**Lab 1 : Data Model**

In [5]:
# Module Imports
import mariadb
# Execption used to stop the Notebook cell execution politely
class StopExecution(Exception):
    def _render_traceback_(self):
        pass
# Connect to the server and return a Connection object for the db_name database.
def connectToDB(db_name):
    try:
        return mariadb.connect(
            user="root",
            password="we4reDATA",
            host="localhost", # Use of localhost because 1/ we don't have
#acces to the docker local network 2/ we have made a port redirection from the
#mariadb server to the host
            port=3306,
            database=db_name
        )
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        raise StopExecution
conn = connectToDB("flight_reservation")

In [6]:
def showTablesAndColumns(conn):
    cur = conn.cursor()
    # Query the tables
    cur.execute("SHOW TABLES;")
    # For each table
    for table in cur.fetchall():
        print("**",table[0],"**")
        # Defining the output format {:16} indicate at least 16 characters, add spaces if necessary
        outputFormat="{:16}\t{:10}\t{}\t{}\t{}\t{}"
        # Printing the headers
        print(outputFormat.format("Field","Type","Null","Key","Default","Extra"))
        # Query the columns
        cur.execute('SHOW COLUMNS FROM {};'.format(table[0]))
        # For each column
        for attribute in cur.fetchall():
            # Print the attribute (*attribute give the content of the tuple to the format function)
            print(outputFormat.format(*attribute))
showTablesAndColumns(conn)

** Booking **
Field           	Type      	Null	Key	Default	Extra
id_booking      	int(11)   	NO	PRI	None	
client_name     	varchar(50)	NO		None	
quantity        	int(11)   	NO		None	
id_flight       	varchar(20)	NO	MUL	None	
** Flight **
Field           	Type      	Null	Key	Default	Extra
id_flight       	varchar(20)	NO	PRI	None	
maximum_capacity	int(11)   	NO		None	
current_capacity	int(11)   	NO		None	
** Show_flight **
Field           	Type      	Null	Key	Default	Extra
id_flight       	varchar(20)	NO		None	
maximum_capacity	int(11)   	NO		None	
current_capacity	int(11)   	NO		None	
** show booking **
Field           	Type      	Null	Key	Default	Extra


ProgrammingError: You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'show booking' at line 1

In [ ]:
def add_flight(conn, id_flight, current_capacity, max_capacity):
    cur = conn.cursor()
    # Insert the flight into the flights table
    cur.execute("INSERT INTO Flight (id_flight, current_capacity, maximum_capacity) VALUES (?, ?, ?)", (id_flight, current_capacity, max_capacity))
    # Commit the changes to the database
    conn.commit()
add_flight(conn,1,0,50)

In [ ]:
def add_booking(conn, id_flight, id_user, quantity, clef):
    cur = conn.cursor()
    cur.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE")
    # Insert the booking into the bookings table
    if not test_flight_exists(conn, id_flight) :
        return False, "Invalid"

    if not test_available_seats(conn, id_flight, quantity) :
        return False, "Full"

    try:
        cur.execute("INSERT INTO Booking (id_booking, client_name, quantity, id_flight) VALUES (?, ?, ?, ?)", (clef, id_user, quantity, id_flight))
        conn.commit()
    except mariadb.Error as e:
        print(f"Error inserting booking: {e}")
        conn.rollback()
        return True, "Error"
    
    try :
        cur.execute("UPDATE Flight SET current_capacity = current_capacity + ? WHERE id_flight = ?", (quantity, id_flight))
        return False, "Reserved"
    except mariadb.Error as e:
        print(f"Error updating flight capacity: {e}")
        conn.rollback()
        return True, "Error"
    # Commit the changes to the database

def test_flight_exists(db_connection, flight_id):
    query = "SELECT * FROM Flight WHERE id_flight = %s"
    cursor = db_connection.cursor()
    cursor.execute(query, (flight_id,))
    row = cursor.fetchone()

    if row:
        return True  # Flight exists
    else:
        return False

def test_available_seats(db_connection, flight_id, quantity):
    query = "SELECT * FROM Booking WHERE id_flight = %s"
    cur = db_connection.cursor()
    cur.execute(query, (flight_id,))
    rows = cur.fetchall()

    total_seats = 0

    if not (test_flight_exists(db_connection, flight_id)) :
        return "Error : Flight don't exist"
    cur.execute("SELECT maximum_capacity FROM Flight WHERE id_flight = %s", (flight_id,)) 
    max = cur.fetchone()[0]
    cur.execute("SELECT current_capacity FROM Flight WHERE id_flight = %s", (flight_id,))
    curent = cur.fetchone()[0]
    available_seats =  max - curent

    if available_seats < quantity:
        return False
    else:
        return True

In [ ]:
def cleanDB(id_flight):
    conn = connectToDB("flight_reservation")
    cur = conn.cursor()
    cur.execute("DELETE FROM Booking WHERE id_flight=?;",(id_flight,))
    cur.execute("UPDATE Flight SET current_capacity=0 WHERE id_flight=?;",(id_flight,))
    conn.commit() # Without commit the modification are not applied to the database

In [ ]:
import random
def processBooking(id_flight, client_name, quantity):
    print("Processing Flight",id_flight,"for",client_name,":", quantity,"seats.")
    retry,status = add_booking(conn, id_flight, client_name, quantity, random.randint(1,1000000))
    print(client_name,":",status, "Should retry:",retry)
    while retry :
        print("Retry Flight",id_flight,"for",client_name,":", quantity,"seats.")
        retry,status = addBooking(id_flight,client_name,quantity)
        print(client_name,":",status)

In [ ]:
import threading
import random
import time
# Test parameters
id_flight="LH6795"
base_name="Client "
# Clean the db for the test
#cleanDB(id_flight)
# Creating several Thread representing several client that try to book simultaneously a random number of seat
jobs=[]
for i in range(5):
    jobs.append(threading.Thread(target=processBooking,args=(id_flight,base_name+str(i), random.randint(1,4))))
# Start each job, giving 3s of delay between each
for job in jobs:
    job.start()
    #time.sleep(3)
    # Wait for all the thread to finish
for job in jobs:
    job.join()
    print("Finished")

Exception in thread Thread-337 (processBooking):
Traceback (most recent call last):
  File "c:\Users\coren\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-338 (processBooking):
Traceback (most recent call last):
  File "c:\Users\coren\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-339 (processBooking):
Traceback (most recent call last):
  File "c:\Users\coren\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-340 (processBooking):
Traceback (most recent call last):
  File "c:\Users\coren\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\coren\anaconda3\Lib\threading.py", line 1012, in run
Exception in thread Thread-341 (processBooking):
Traceback (most recent call last):
  File "c:\Users\coren\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\coren\anaconda3\Lib\threading.py", line 1012, in

Processing Flight LH6795 for Client 0 : 2 seats.
Processing Flight LH6795 for Client 1 : 2 seats.
Processing Flight LH6795 for Client 2 : 1 seats.
Processing Flight LH6795 for Client 3 : 4 seats.
Processing Flight LH6795 for Client 4 : 1 seats.
Finished
Finished
Finished
Finished
Finished


In [ ]:
conn = connectToDB("flight_reservation")
cur = conn.cursor()
cur.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE")
cur.execute("SELECT COUNT(*) FROM Flight WHERE id_flight=?",(id_flight,))